In [1]:
__version__ = "2.0"

# Stock Visualization Analisys

In [1]:
# common.py
import common
from common import symmetric_simple_moving_average, calc_total_days

# parameters.py
from parameters import get_parameters_pnl

# interpolation.py
import interpolation as itpl
from interpolation import aggregate

# visualization_functions.py
import visualization_functions as vf
from visualization_functions import *

# Standard Library
from datetime import datetime, timedelta
import calendar
from collections import defaultdict
from time import perf_counter

# General
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
libraries = [np, pd, itpl, vf, common]

for lib in libraries:
    print(f"{lib.__name__:>23}: {lib.__version__}")

                  numpy: 1.19.2
                 pandas: 1.1.3
          interpolation: 1.0
visualization_functions: 1.0
                 common: 1.0


In [4]:
plt.style.use("bmh")
plt.rcParams['figure.figsize'] = (16, 10 / 2)

np.random.seed(42)

# Simulation Execution

In [ ]:
parameters, months = get_parameters_pnl()
replicates = 20

start_time = perf_counter()
aggregated_df, dfs_interpolates = aggregate(parameters,replicates)
elapsed = perf_counter() - start_time

df_bytes = np.sum([df.memory_usage() for df in dfs_interpolates])
df_megabytes = df_bytes / 1024 / 1024

print(f"{replicates:3d} Replicates | Mean Size: {df_megabytes:7.2f}MB | Mean Execution Time {elapsed:5.1f}s")

# Aggregations

In [ ]:
# Group By Mean
df_means = aggregated_df.mean()
df_std = aggregated_df.std()
df_minus2std = df_means - 2 * df_std
df_plus2std = df_means + 2 * df_std

# Group By Quantiles
quantile_range = 0.95
df_median = aggregated_df.median()
df_quantile_025 = aggregated_df.quantile((1 - quantile_range) / 2) #0.025
df_quantile_975 = aggregated_df.quantile(1 - (1 - quantile_range) / 2) #0.975

In [ ]:
df_quantile_025

# Shadow Plots

In [ ]:
shadow_plot(df_means['capital'], df_minus2std['capital'], df_plus2std['capital'], 'Mean', '+-2 STD', 'Capital with 2 STD Confidence', 'darkcyan')
plt.show()

shadow_plot(df_median['capital'], df_quantile_025['capital'], df_quantile_975['capital'], 'Median', '95% Range', 'Capital with 95% Confidence', 'firebrick')
plt.show()

shadow_plot(df_means['capital'], df_minus2std['capital'], df_plus2std['capital'], 'Mean', '+-2 STD', 'Capital with Confidence', 'darkcyan')
shadow_plot(df_median['capital'], df_quantile_025['capital'], df_quantile_975['capital'], 'Median', '95% Range', 'Capital with Confidence', 'firebrick')
plt.show()

# All replicates plot

In [ ]:
for df in dfs_interpolates:
    df['capital'].plot(linewidth=1)

plt.title(f'{replicates} simulations plot')
plt.xlabel('Days')
plt.ylabel('Capital ($)')
plt.show()

# Histogram of slices of a specific times

In [ ]:
indexes = (np.linspace(1000, len(dfs_interpolates[0])-1000, 5)).astype(int)
for i in indexes:
    slice_ = [df['capital'].iloc[i] for df in dfs_interpolates]
    plt.hist(slice_, label=str(df['capital'].index[i].date()))
    
plt.title('Histograms of slices of capital')
plt.xlabel('Capital ($)')
plt.legend()
plt.show()

# Flow Plots

In [ ]:
initial_stock_cost = parameters['unit_cost'] * parameters['stock_level']
flow_plot(df_means['stock'],df_means['stock_floor'],df_means['stock_ceil'],'Instant Stock Flow Mean','Stock Level','s Level','S Level',with_floor=True,with_ceil=True)
flow_plot(df_means['capital']-initial_stock_cost,df_means['stock_floor'],df_means['stock_ceil'],'Capital Flow Mean','Capital ($)','s Level','S Level',with_floor=False,with_ceil=False)
flow_plot(df_means['price'],df_means['price_floor'],df_means['price_ceil'],'Price Flow Mean','Price ($)','Minimum','Maximum',with_floor=True,with_ceil=True)

# Agregate with period plot - Mean/Median

In [ ]:
aggregate_with_period_low_high(df_means['stock'], df_minus2std['stock'], df_plus2std['stock'], 'D', 'Daily Stock - Mean')
aggregate_with_period_low_high(df_median['stock'], df_quantile_025['stock'], df_quantile_975['stock'], 'D', 'Daily Stock - Median')
aggregate_with_period_low_high(df_median['stock'], df_quantile_025['stock'], df_quantile_975['stock'], 'M', 'Monthly Stock - Median')
aggregate_with_period_low_high(df_median['capital'], df_quantile_025['capital'], df_quantile_975['capital'], 'D', 'Daily Capital')

# Time Series Plot Shadow

In [ ]:
df_means['arrivals']

In [ ]:
plot_time_series_shadow(df_means['arrivals'], df_minus2std['arrivals'], df_plus2std['arrivals'], 'D', 'arrivals', "Daily Arrivals Mean")
plot_time_series_shadow(df_means['sales'], df_minus2std['sales'], df_plus2std['sales'], 'D', 'sales', "Daily Sales Mean")

# Replicas Scatter plot

In [ ]:
for df in dfs_interpolates:
    plot_scatter(df['losses'], 'D', "Daily Losses", 'darkred', show=False)
plt.show()

for df in dfs_interpolates:
    plot_scatter(df['orders'], 'D', "Daily Losses", 'forestgreen', show=False)
plt.show()

# High/Middle/Low Scatter Plots

In [ ]:
plot_scatter(df_quantile_975['losses'], 'M', "Daily Losses", 'rebeccapurple', show=False)
plot_scatter(df_median['losses'], 'M', "Daily Losses", 'forestgreen', show=False)
plot_scatter(df_quantile_025['losses'], 'M', "Daily Losses", 'salmon', show=False)
plt.show()

plot_scatter(df_quantile_975['orders'], 'M', "Daily Losses", 'rebeccapurple', show=False)
plot_scatter(df_median['orders'], 'M', "Daily Losses", 'forestgreen', show=False)
plot_scatter(df_quantile_025['orders'], 'M', "Daily Losses", 'salmon', show=False)
plt.show()